# Projeto 2 - Ciência dos Dados

Nome: Beatriz Muniz de Castro e Silva

Nome: Nicole Sarvasi Alves da Costa

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [6]:
%%capture

#Instalando o tweepy
!pip install tweepy
#Instalando o pacote emoji para limpar mensagens
!pip install emoji

In [7]:
#importando os pacotes necessários
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import functools
import operator
import numpy as np
import emoji
import re

___
## Autenticando no  Twitter

* Conta: @nicknennis

In [8]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @nickcnennis

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [9]:
#Produto escolhido:
produto = 'bacurau'

#Quantidade mínima de mensagens capturadas:
n = 700
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [10]:
#Cria um objeto para a captura
api = tweepy.API(auth, wait_on_rate_limit=True)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []

#retira os retweets
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode='extended').items():  
    if (not msg.retweeted) and ('RT' not in msg.full_text): 
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

TweepError: Twitter error response: status code = 401

Salvando os dados em uma planilha Excel:

In [15]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [11]:
#leitura do arquivo excel
treino = pd.read_excel("bacurau.xlsx", "Treinamento")
teste = pd.read_excel("bacurau.xlsx", "Teste")

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [12]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    
    text = ' '.join(word for word in text.split() if not word.startswith('https'))
    
    text_split_emoji = emoji.get_emoji_regexp().split(text)
    text_split_whitespace = [substr.split() for substr in text_split_emoji]
    text_split = functools.reduce(operator.concat, text_split_whitespace)
    text = ' '.join(word for word in text_split)
    
    punctuation = '[!-/.:?;@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    nova_linha = '[\n]'
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed = re.sub(nova_linha, " ", text_subbed)
    return text_subbed

In [13]:
treino_t = pd.Series(treino["Treinamento"])
treino_t = treino_t.str.cat()
treino_t = cleanup(treino_t.lower())
lista_relativa = treino_t.split()
frequencia_absoluta = pd.Series(lista_relativa).value_counts()
frequencia_relativa = pd.Series(lista_relativa).value_counts(True)
frequencia_relativa

bacurau         0.056375
de              0.046683
e               0.021431
a               0.019383
é               0.018701
que             0.018564
o               0.015561
eu              0.012149
você            0.010920
do              0.010511
tirei           0.010238
ver             0.009419
pra             0.009419
no              0.008463
assistir        0.008327
um              0.008327
filme           0.007644
não             0.007235
para            0.007098
uma             0.007098
com             0.007098
em              0.006416
da              0.006279
tem             0.005870
mas             0.005460
gente           0.005324
lunga           0.004778
me              0.004778
qual            0.004778
domingas        0.004778
                  ...   
papo            0.000137
vivam           0.000137
pantanal        0.000137
🥰               0.000137
rasteira        0.000137
receberá        0.000137
del             0.000137
sdds            0.000137
tirar           0.000137


In [28]:
rel = treino.index[treino.Relevância == 1]
treino_r = treino.loc[rel, "Treinamento"]  #tweets relevantes

irel = treino.index[treino.Relevância == 0]
treino_ir = treino.loc[irel, "Treinamento"]  #tweets irrelevantes


#frequência das palavras nos tweets relevantes
treino_rt = treino_r.str.cat()
treino_rt = cleanup(treino_rt.lower())
lista_relevante = treino_rt.split()
frequencia_absoluta = pd.Series(lista_relevante).value_counts()
frequencia_relevantes = pd.Series(lista_relevante).value_counts(True)
#frequencia_relevantes


#frequência das palavras nos tweets irrelevantes
treino_irt = treino_ir.str.cat()
treino_irt = cleanup(treino_irt.lower())
lista_irrelevante = treino_irt.split()
frequencia_absoluta = pd.Series(lista_irrelevante).value_counts()
frequencia_irrelevantes = pd.Series(lista_irrelevante).value_counts(True)
#frequencia_irrelevantes

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**